In [37]:
pip install torch

In [38]:
pip install transformers

In [39]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
#from transformers import BertConfig, BertModel
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
#model = 'bert-base-uncased'.from_pretrained('bert-base-uncased')

In [40]:
SPECIAL_TOKENS_DICT = {
    'pad_token': '<pad>',
    'additional_special_tokens': ['<context>', '<slogan>'],
}

tokenizer.add_special_tokens(SPECIAL_TOKENS_DICT)
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [41]:
import csv, torch
import regex as re
from torch.utils.data import Dataset

class SloganDataset(Dataset):
  def __init__(self, filename, tokenizer, seq_length=64):

    context_tkn = tokenizer.additional_special_tokens_ids[0]
    slogan_tkn = tokenizer.additional_special_tokens_ids[1]
    pad_tkn = tokenizer.pad_token_id
    eos_tkn = tokenizer.eos_token_id

    self.examples = []
    with open(filename, 'r', encoding='utf8', errors='ignore') as csvfile:
      reader = csv.reader(csvfile)
      #reader[1] = reader[1].encode('windows-1252').decode('utf-8')
      for row in reader:
        print("1: ",row)
        emptyString = ""
        row[0] = re.sub(r"\xa0", " ", row[0])
        row[1] = re.sub(r"\xa0", " ", row[1])
        row[0] = re.sub(r"Ã", " ", row[0])
        row[1] = re.sub(r"Ã", " ", row[1])
        row[0] = re.sub(r"Ã‚Â\xa0", ",", row[0])
        row[1] = re.sub(r"Ã‚Â\xa0", ",", row[1])
        row[0] = re.sub(r"ÃƒÂ©", "e", row[0])
        row[1] = re.sub(r"ÃƒÂ©", "e", row[1])
        print("2: ",row)
        # Build the context and slogan segments:
        context = [context_tkn] + tokenizer.encode(row[0], max_length=seq_length//2-1)
        slogan = [slogan_tkn] + tokenizer.encode(row[1], max_length=seq_length//2-2) + [eos_tkn]

        # Concatenate the two parts together:
        tokens = context + slogan + [pad_tkn] * ( seq_length - len(context) - len(slogan) )

        # Annotate each token with its corresponding segment:
        segments = [context_tkn] * len(context) + [slogan_tkn] * ( seq_length - len(context) )

        # Ignore the context, padding, and <slogan> tokens by setting their labels to -1
        labels = [-100] * (len(context)+1) + slogan[1:] + [-100] * ( seq_length - len(context) - len(slogan) )

        # Add the preprocessed example to the dataset
        self.examples.append((tokens, segments, labels))

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item])

In [43]:
slogan_dataset = SloganDataset('/content/slogans.csv', tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Streaming output truncated to the last 5000 lines.
1:  ['Audi automobiles', 'Everyone dreams of an Audi.']
2:  ['Audi automobiles', 'Everyone dreams of an Audi.']
1:  ['Volkswagen, the German car brand', 'Volkswagen. Drivers wanted.']
2:  ['Volkswagen, the German car brand', 'Volkswagen. Drivers wanted.']
1:  ['Volkswagen, the German car brand', 'For the love of the car.']
2:  ['Volkswagen, the German car brand', 'For the love of the car.']
1:  ['Volkswagen, the German car brand', 'Relieves gas pains.']
2:  ['Volkswagen, the German car brand', 'Relieves gas pains.']
1:  ['Volkswagen, the German car brand', 'Surprisingly ordinary prices']
2:  ['Volkswagen, the German car brand', 'Surprisingly ordinary prices']
1:  ['Peugeot, the French automobiles', 'Peugeot. Live the pleasure.']
2:  ['Peugeot, the French automobiles', 'Peugeot. Live the pleasure.']
1:  ['Peugeot, the French automobiles', 'The drive of your life.']
2:  ['Peugeot, the French automobiles', 'The drive of your life.']
1:  [

In [44]:
import math, random
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

# Create data indices for training and validation splits:

indices = list(range(len(slogan_dataset)))

random.seed(42)
random.shuffle(indices)

split = math.floor(0.1 * len(slogan_dataset))
train_indices, val_indices = indices[split:], indices[:split]

# Build the PyTorch data loaders:

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(slogan_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(slogan_dataset, batch_size=64, sampler=val_sampler)

In [45]:
import numpy as np
from tqdm import tqdm
def fit(model, optimizer, train_dl, val_dl, epochs=1, device=torch.device('cpu')):

  for i in range(epochs):

    print('\n--- Starting epoch #{} ---'.format(i))

    model.train()

    # These 2 lists will keep track of the batch losses and batch sizes over one epoch:
    losses = []
    nums = []

    for xb in tqdm(train_dl, desc="Training"):
      # Move the batch to the training device:
      print(len(xb))
      inputs = xb.to(device)
      print(inputs[:,1,:])
      # Call the model with the token ids, segment ids, and the ground truth (labels)
      outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])

      # Add the loss and batch size to the list:
      loss = outputs[0]
      losses.append(loss.item())
      nums.append(len(xb))

      loss.backward()

      optimizer.step()
      model.zero_grad()

    # Compute the average cost over one epoch:
    train_cost = np.sum(np.multiply(losses, nums)) / sum(nums)


    # Now do the same thing for validation:

    model.eval()

    with torch.no_grad():
      losses = []
      nums = []

      for xb in tqdm(val_dl, desc="Validation"):
        inputs = xb.to(device)
        outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
        losses.append(outputs[0].item())
        nums.append(len(xb))

    val_cost = np.sum(np.multiply(losses, nums)) / sum(nums)

    print('\n--- Epoch #{} finished --- Training cost: {} / Validation cost: {}'.format(i, train_cost, val_cost))

In [46]:
from transformers import AdamW

# Move the model to the GPU:
device = torch.device('cuda')
model.to(device)

# Fine-tune GPT2 for two epochs:
optimizer = AdamW(model.parameters())
fit(model, optimizer, train_loader, val_loader, epochs=2, device=device)

Training:   0%|          | 0/287 [00:00<?, ?it/s]


--- Starting epoch #0 ---
32
tensor([[50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        ...,
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259]], device='cuda:0')


Training:   0%|          | 1/287 [00:00<02:50,  1.68it/s]

32
tensor([[50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        ...,
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259]], device='cuda:0')


Training:   1%|          | 2/287 [00:01<02:44,  1.74it/s]

...

32
tensor([[50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        ...,
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259]], device='cuda:0')


Training: 100%|█████████▉| 286/287 [02:41<00:00,  1.77it/s]

32
tensor([[50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        ...,
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259],
        [50258, 50258, 50258,  ..., 50259, 50259, 50259]], device='cuda:0')


Validation: 100%|██████████| 16/16 [00:06<00:00,  2.54it/s]


--- Epoch #1 finished --- Training cost: 2.725821315619173 / Validation cost: 3.279965400695801


In [48]:
torch.save(model, '/content/model_slogan.pkl')

In [49]:
# Sampling functions with top k and top p from HuggingFace:

import torch.nn.functional as F
from tqdm import trange


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (batch size x vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 1:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.5:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # scatter sorted tensors to original indexing
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits


# From HuggingFace, adapted to work with the context/slogan separation:
def sample_sequence(model, length, context, segments_tokens=None, num_samples=1, temperature=1, top_k=0, top_p=0.0, repetition_penalty=1.0,
                    device='cpu'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context

    with torch.no_grad():
        for _ in trange(length):

            inputs = {'input_ids': generated}
            if segments_tokens != None:
              inputs['token_type_ids'] = torch.tensor(segments_tokens[:generated.shape[1]]).unsqueeze(0).repeat(num_samples, 1)


            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet/CTRL (cached hidden-states)
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # repetition penalty from CTRL (https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            if temperature == 0: # greedy sampling:
                next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
            else:
                next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token), dim=1)
    return generated

In [ ]:
context = "Matrimonial"

context_tkn = tokenizer.additional_special_tokens_ids[0]
slogan_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [context_tkn] + tokenizer.encode(context)

segments = [slogan_tkn] * 64
segments[:len(input_ids)] = [context_tkn] * len(input_ids)

input_ids += [slogan_tkn]

# Move the model back to the CPU for inference:
model.to(torch.device('cpu'))

# Generate 20 samples of max length 20
generated = sample_sequence(model, length=20, context=input_ids, segments_tokens=segments, num_samples=30)

print('\n\n--- Generated Slogans ---\n')

for g in generated:
  slogan = tokenizer.decode(g.squeeze().tolist())
  slogan = slogan.split('<|endoftext|>')[0].split('<slogan>')[1]
  print(slogan)

 45%|████▌     | 9/20 [00:10<00:14,  1.31s/it]